
**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Edición 2023**

---

# Trabajo práctico entregable - Parte 2

In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats


sns.set_context('talk')

## Lectura del dataset

En la notebook 00 se explican los detalles de la siguiente sección.

In [ ]:
url = 'https://raw.githubusercontent.com/DiploDatos/AnalisisyVisualizacion/master/sysarmy_survey_2022_processed.csv'
df = pd.read_csv(url)

In [ ]:
df[:3]

,work_country,work_province,work_dedication,work_contract_type,salary_monthly_BRUTO,salary_monthly_NETO,numero,salary_in_usd,salary_last_dollar_value,salary_pay_cripto,...,profile_studies_level_state,profile_career,profile_university,profile_boot_camp,profile_boot_camp_carrer,work_on_call_duty,salary_on_call_duty_charge,work_on_call_duty_charge_type,profile_age,profile_gender
0,Argentina,Catamarca,Full-Time,Staff (planta permanente),300000.0,245000.0,True,NaN,NaN,NaN,...,Completo,Licenciatura en redes y comunicación de datos,UP - Universidad de Palermo,NaN,NaN,NaN,NaN,NaN,35,Varón Cis
1,Argentina,Chaco,Full-Time,Remoto (empresa de otro país),900000.0,850000.0,True,Cobro todo el salario en dólares,300,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,Varón Cis
2,Argentina,Chaco,Full-Time,Staff (planta permanente),120000.0,115000.0,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,Varón Cis


In [ ]:
df.profile_gender.unique()

array(['Varón Cis', 'Varón cis', 'Mujer Cis', 'Mujer', 'Varón Trans',
       'Queer', 'No binarie', 'Mujer cis', 'Mujer Trans', 'mujer', 'Gay',
       'Fluido', 'Bigénero', 'Femenino'], dtype=object)

In [ ]:
df.loc[:,'profile_g'] = df.profile_gender.replace({'Varón Cis': 'Varón cis','Mujer': 'Mujer cis','Mujer Cis': 'Mujer cis','Femenino': 'Mujer cis','mujer': 'Mujer cis','Mujer':'Mujer cis','Queer':'Diversidades','Varón Trans':'Diversidades','No binarie':'Diversidades','Mujer Trans':'Diversidades','Fluido':'Diversidades','Bigénero':'Diversidades','Gay':'Diversidades'}).fillna(False)

In [ ]:
df.profile_g.unique()

array(['Varón cis', 'Mujer cis', 'Diversidades'], dtype=object)

In [ ]:
df[['profile_g', 'salary_monthly_NETO']].groupby('profile_g').describe()

salary_monthly_NETO                                               \
                           count           mean            std  min       25%   
profile_g                                                                       
Diversidades                98.0  261873.543041  356719.368150  0.0   90500.0   
Mujer cis                  929.0  188903.430667  184912.713803  0.0   91000.0   
Varón cis                 4331.0  296252.343889  433193.328154  0.0  125000.0   

                                              
                   50%       75%         max  
profile_g                                     
Diversidades  174448.5  262250.0   1950000.0  
Mujer cis     160000.0  237003.0   1878500.0  
Varón cis     212146.0  307988.0  15000000.0

In [ ]:
alpha = 0.05

In [ ]:
is_man = df.profile_g == 'Varón cis'

groupA = df[(df.salary_monthly_NETO > 1000)  & is_man].salary_monthly_NETO
groupB = df[(df.salary_monthly_NETO > 1000) & ~is_man].salary_monthly_NETO

groupA = df[(df.salary_monthly_NETO > 1000) & is_man]
groupB = df[(df.salary_monthly_NETO > 1000) & ~is_man]



## Ejercicio 1: Estimación

**Consigna:**  Calcular una estimación puntual y un intervalo de confianza de nivel (1-alpha) para la resta entre la media del salario Neto para Hombres menos la media del salario Neto para otros géneros(diferencia de las medias entre el grupoA y grupoB).
¿Cómo se relaciona este intervalo de confianza con el test de hipótesis?

In [ ]:
df_concat = pd.concat([groupA, groupB])
df_concat['salary_monthly_NETO'].groupby(is_man).describe()

,count,mean,std,min,25%,50%,75%,max
profile_g,,,,,,,,
False,891.0,225741.927003,208096.345457,1400.0,116750.0,180000.0,250000.0,1950000.0
True,3975.0,322777.029197,442612.347688,1100.0,147106.5,225000.0,322827.5,15000000.0


In [ ]:
#Cálculo de la media de salario neto para hombres y para el resto de géneros
Media_GA = groupA['salary_monthly_NETO'].mean()
Media_GB = groupB['salary_monthly_NETO'].mean()
print(Media_GA,Media_GB)


322777.02919748425 225741.927003367


In [ ]:
std_GA = groupA['salary_monthly_NETO'].std()
std_GB = groupB['salary_monthly_NETO'].std()
print(std_GA,std_GB)

442612.3476877616 208096.345457438


In [ ]:
N_GA = groupA['salary_monthly_NETO'].count()
N_GB = groupB['salary_monthly_NETO'].count()
print(N_GA,N_GB)

3975 891


Intervalo de confianza para la media de nivel

In [ ]:
alpha = 0.05  # Nivel de confianza del 95%
nA = len(groupA)
nB = len(groupB)
t_crit = stats.t.ppf(1 - alpha/2, nA + nB - 2) #se obtiene el valor crítico de t para un nivel de confianza del 95% y nA+nB-2 grados de libertad (t_crit) usando la función ppf de la biblioteca scipy.stats.
t_crit

1.9604518241004518

In [ ]:
#intervalo de confianza
resta_medias_liminf=(Media_GA-Media_GB)-t_crit * ((std_GA/np.sqrt(N_GA))+(std_GB/np.sqrt(N_GB)))
resta_medias_limsup=(Media_GA-Media_GB)+t_crit * ((std_GA/np.sqrt(N_GA))+(std_GB/np.sqrt(N_GB)))
print(resta_medias_liminf,resta_medias_limsup)

69604.89477818095 124465.30961005358


In [ ]:
#Estimador puntual
Media_resta=Media_GA-Media_GB
Media_resta

97035.10219411727

**Con una confianza del 95% la diferencia de medias poblacionales se encontrará entre 69604 y 124465.**

In [ ]:
long_IC=(resta_medias_limsup-resta_medias_liminf)
long_IC

54860.41483187262

**La relación entre este intervalo de confianza y el test de hipótesis es que el intervalo de confianza nos da información sobre el rango de valores plausibles para la diferencia de las medias entre los dos grupos, mientras que el test de hipótesis nos dice si la diferencia de las medias es estadísticamente significativa o no. Si el intervalo de confianza no incluye cero, podemos concluir que la diferencia de las medias es significativamente distinta de cero al nivel de confianza dado. Por otro lado, si el intervalo de confianza incluye cero, no podemos rechazar la hipótesis nula de que no hay diferencia significativa entre las medias al nivel de confianza dado.
Entonces ambos  utilizan enfoques diferentes para responder la pregunta de interes.**

## Ejercicio 2: Test de hipótesis



### 2.1 Formalización

Describir formalmente los distintos componentes de un test de hipótesis para comprobar si la distribución de los salarios es distinta entre los grupos A y B.

**Hipótesis Nula**

$H_0=...$ --> **La media de la distribución de salarios en el grupo A es igual a la media de la distribución de salarios en el grupo B.**

**Estadístico (Pivote)**
  * **Identificar el estadístico :** El estadístico de prueba se utiliza para evaluar la hipótesis nula. En este caso, el estadístico de prueba podría ser la diferencia de medias entre los grupos A y B:
Pivote = (Media_GA) - (Media_GB)
  * **Escribir qué distribución tiene bajo $H_0$**: Se necesita determinar la distribución de probabilidad del estadístico de prueba bajo la hipótesis nula. En este caso, la distribución de probabilidad del estadístico de prueba sería una distribución t de Student con (nA + nB - 2) grados de libertad, donde nA y nB son el número de observaciones en los grupos A y B, respectivamente.


### 2.2 P-valor

1. Calcule el p-valor y decida si rechazar o no la hipótesis nula.
2. Interprete el resultado.

Links útiles:
* [Test de hipótesis usando scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
* [Test de Welch](http://daniellakens.blogspot.com/2015/01/always-use-welchs-t-test-instead-of.html)

In [ ]:
nA = 3975
nB = 891
muA = 322777.02919748425
muB = 225741.927003367
stdA = 442612.3476877616
stdB = 208096.345457438
alpha = 0.05 # Nivel de confianza del 95%

# Calcular la varianza combinada de las muestras
Sp = ((nA - 1)*stdA**2 + (nB - 1)*stdB**2) / (nA + nB - 2)

# Calcular el estadístico t
t = (muA - muB) / (Sp * (1/nA + 1/nB))**0.5

# Calcular los grados de libertad
df = nA + nB - 2

# Calcular el valor crítico de t
t_critico = stats.t.ppf(1 - alpha/2, df)

# Calcular el p-valor
p_valor = 2 * (1 - stats.t.cdf(abs(t), df))

# Imprimir los resultados
print("Estadístico t: ", t)
print("Valor crítico de t: ", t_critico)
print("p-valor: ", p_valor)

# Interpretar el resultado
if abs(t) > t_critico:
    print("Se rechaza la hipótesis nula.")
else:
    print("No se puede rechazar la hipótesis nula.")

if p_valor < alpha:
    print("La diferencia entre las medias de los grupos A y B es estadísticamente significativa.")
else:
    print("La diferencia entre las medias de los grupos A y B no es estadísticamente significativa.")

Estadístico t:  6.387301602375004
Valor crítico de t:  1.9604518241004518
p-valor:  1.8459234141232628e-10
Se rechaza la hipótesis nula.
La diferencia entre las medias de los grupos A y B es estadísticamente significativa.


**La hipótesis nula plantea que no hay diferencia significativa entre la media de los salarios del grupo A y el grupo B. Por lo tanto, si el p-valor es menor que el nivel de significancia (0.05), entonces se rechaza la hipótesis nula y se concluye que hay evidencia suficiente para afirmar que las medias de los salarios entre el grupo A y el grupo B son diferentes y existe una diferencia significativa entre ellos.**

**En nuestro caso el p-valor es menor que el alpha , por lo que se puede concluir que hay evidencia suficiente para afirmar que las medias de los salarios entre el grupo A y el grupo B son diferentes. Por lo tanto, se rechaza la hipótesis nula y se concluye que hay una diferencia significativa entre las medias de los salarios entre ambos grupos.**



### [Opcional] 2.3 Potencia del test

Nuestra muestra, ¿era lo suficientemente grande para detectar si existe o no una diferencia entre los grupos?

1. Utilice la función `tt_ind_solve_power` para calcular el tamaño necesario de la muestra para un poder estadístico de 0.8, 0.9 y 0.95, asumiendo una significancia estadística de 0.05.
2. ¿Cómo intepretan el poder estadístico de un test? Dado su conocimiento de dominio sobre los datos, ¿les parece que esta muestra es lo suficientemente grande para ser representativo de la tendencia general? ¿y para utilizarlo en un juicio penal contra una empresa XX por una causa de discriminación?

[Documentación](https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html)

NOTA: este análisis debería hacerse ANTES de recolectar los datos.

**1. Para analizar si la muestra es lo suficientemente grande para ser representativa de la tendencia general,va a dependerá de la población de interés y de la variabilidad en los datos. Si la población de interés es grande y la variabilidad en los datos es baja, una muestra de 3975 y 891 puede ser lo suficientemente grande para ser representativa de la tendencia general. Sin embargo, si la población de interés es pequeña y/o la variabilidad en los datos es alta, se necesitará una muestra más grande para que sea representativa de la población.**


**2. Para utilizar una muestra así en un juicio penal por discriminación, hay que considerar que todos los factores que influyen en los resultados de la muestra y no se esten teniendo en cuenta para el análisis estadistico.**

In [ ]:
from statsmodels.stats.power import tt_ind_solve_power

In [ ]:
effect_size = (muA - muB) / stdB
print(effect_size)
# nobs1=None  - What we want to know
alpha = 0.05
ratio = nB/nA

sample_size_08 = tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=0.8, ratio=ratio)
sample_size_09 = tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=0.9, ratio=ratio)
sample_size_095 = tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=0.95, ratio=ratio)

print(f"Tamaño necesario de muestra para un poder estadístico de 0.8: {sample_size_08:.0f}")
print(f"Tamaño necesario de muestra para un poder estadístico de 0.9: {sample_size_09:.0f}")
print(f"Tamaño necesario de muestra para un poder estadístico de 0.95: {sample_size_095:.0f}")

0.4662989250523089
Tamaño necesario de muestra para un poder estadístico de 0.8: 199
Tamaño necesario de muestra para un poder estadístico de 0.9: 265
Tamaño necesario de muestra para un poder estadístico de 0.95: 328


**El resultado de la función "tt_ind_solve_power" será el tamaño de muestra necesario para alcanzar la potencia deseada. 199, seria el tamaño de muestra necesario para detectar una diferencia de medias específica entre dos muestras con una potencia estadística determinada(80%) y un nivel de significancia dado(5%)**




## Ejercicio 3: Comunicación y visualización

**Consigna:** Seleccionen un resultado que les parezca relevante a partir de alguno de los ejercicios del entregable. Diseñe e implemente una comunicación en base a este mensaje, en un archivo PDF.

Elija las palabras y visualización más adecuada para que la comunicación sea entendible, efectiva y se dapte a UNA de las siguientes situaciones:

1. Una sección en un artículo de difusión a presentar como parte de una organización sin fines de lucro.
No más de 1 página A4 (o dos si los gráficos son muy grandes).
  1. Ejemplo: Alguna de las secciones [Los ecosistemas de emprendimiento de América Latina y el Caribe frente al COVID-19: Impactos, necesidades y recomendaciones](https://publications.iadb.org/es/los-ecosistemas-de-emprendimiento-de-america-latina-y-el-caribe-frente-al-covid-19-impactos-necesidades-y-recomendaciones), por ejemplo la sección *2.2. Reacciones de los emprendedores*.
  2. Ejemplo: Alguna de las secciones de [The state of gender pay gap in 2021](https://www.payscale.com/data/gender-pay-gap?tk=carousel-ps-rc-job)
  3. Puntos clave:
    1. Simpleza de los gráficos.
    2. Comunicación en lenguaje simple a personas que no son necesariamente expertos de dominio.
    2. Selección de UNA oración sobre la que se hace énfasis.
    3. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

2. Una publicación científica o reporte técnico interno. No más de una página A4:
  2. Ejemplo: La sección de resultados de [IZA DP No. 12914: The Impact of a Minimum Wage Change on the Distribution of Wages and Household Income](https://www.iza.org/publications/dp/12914/the-impact-of-a-minimum-wage-change-on-the-distribution-of-wages-and-household-income).
  2. Ejemplo: Alguna de las secciones de [Temporary reduction in daily global CO2 emissions during the COVID-19 forced confinement](https://www.nature.com/articles/s41558-020-0797-x)
  3. Puntos clave:
    3. Nivel de detalle técnico requerido. Es necesario justificar la validez del análisis.
    4. La idea presentada puede ser más compleja. Pueden asumir que la audiencia tiene conocimiento técnico y va a analizar las visualizaciones en detalle.
    5. Pueden presentar más en detalle las limitaciones del análisis (significancia estadística, etc.)
    2. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

3. Un tweet (o post de LinkedIn) para la cuenta de su empresa consultora que hace análisis de datos. El objetivo es promocionar un análisis de datos abiertos que van a incluir en su portfolio:
  1. Ejemplo: [Comparación vacunas covid](https://twitter.com/infobeautiful/status/1381577746527236098?s=20)
  2. Ejemplo: [Tweet del BID](https://twitter.com/el_BID/status/1388508583944507396?s=20). Lo valioso de este tweet es que usaron un único número para transmitir un mensaje. Puede ser algo así, o con un gráfico muy simple.
  3. Ejemplo: [Cambio climático](https://twitter.com/UNFCCC/status/1387732156190011394?s=20) Es un muy buen ejemplo, excepto que el gráfico no se lee nada y hay que entrar a la publicación original.
  3. Ejemplo: [¿Cuánto están los programadores en las empresas?](https://www.linkedin.com/posts/denis-rothman-0b034043_tech-career-work-activity-6793861923269054464-gS6y) (No verificamos la veracidad o seriedad de la fuente).
  4. Puntos clave:
    1. Su audiencia no va a mirar la visualización por más de unos segundos, y no tiene conocimiento técnico.
  3. Tienen que incluir además una *breve* descripción de cómo obtuvieron los datos que están presentando, que no entraría en el tweet.
